# Optimizing Production Capacity for Giant Motor Company Using Mixed Integer Linear Programming (MILP)
## Giant Motor Company

This case deals with strategic planning issues for a large company. The main issue is planning the company’s production capacity for the coming year. At issue is the overall level of capacity and the type of capacity—for example, the degree of flexibility in the manufacturing system. The main tool used to aid the company’s planning process is a mixed integer linear programming (MILP) model. A mixed integer program has both integer and continuous variables.

### Problem Statement

The Giant Motor Company (GMC) produces three lines of cars for the domestic (U.S.) market: Lyras, Libras, and Hydras. The Lyra is a relatively inexpensive subcompact car that appeals mainly to first-time car owners and to households using it as a second car for commuting. The Libra is a sporty compact car that is sleeker, faster, and roomier than the Lyra. Without any options, the Libra costs slightly more than the Lyra; additional options increase the price. The Hydra is the luxury car of the GMC line. It is significantly more expensive than the Lyra and Libra, and it has the highest profit margin of the three cars.

### Retooling Options for Capacity Expansion

Currently GMC has three manufacturing plants in the United States. Each plant is dedicated to producing a single line of cars. In its planning for the coming year, GMC is considering the retooling of its Lyra and/or Libra plants. Retooling either plant would represent a major expense for the company. The retooled plants would have significantly increased production capacities. Although having greater fixed costs, the retooled plants would be more efficient and have lower marginal production costs—that is, higher marginal profit contributions. In addition, the retooled plants would be flexible—they would have the capability of producing more than one line of cars.

The retooled Lyra and Libra plants are prefaced by the word new. The fixed costs and capacities in Table 6.9 are given on an annual basis. A dash in the profit margin section indicates that the plant cannot manufacture that line of car. For example, the new Lyra plant would be capable of producing both Lyras and Libras but not Hydras. The new Libra plant would be capable of producing any of the three lines of cars. Note, however, that the new Libra plant has a slightly lower profit margin for producing Hydras than the Hydra plant. The flexible new Libra plant is capable of producing the luxury Hydra model but is not as efficient as the current Hydra plant that is dedicated to Hydra production.

The fixed costs are annual costs incurred by GMC, independent of the number of cars produced by the plant. For the current plant configurations, the fixed costs include property taxes, insurance, payments on the loan that was taken out to construct the plant, and so on. If a plant is retooled, the fixed costs will include the previous fixed costs plus the additional cost of the renovation. The additional renovation cost will be an annual cost representing the cost of the renovation amortized over a long period.

### Demand for GMC Cars

Short-term demand forecasts have been very reliable in the past and are expected to be reliable in the future.

A quick comparison of plant capacities and demands indicates that GMC is faced with insufficient capacity. Partially offsetting the lack of capacity is the phenomenon of demand diversion. If a potential car buyer walks into a GMC dealer showroom wanting to buy a Lyra but the dealer is out of stock, frequently the salesperson can convince the customer to purchase the better Libra car, which is in stock. Unsatisfied demand for the Lyra is said to be diverted to the Libra. Only rarely in this situation can the salesperson convince the customer to switch to the luxury Hydra model.

From past experience, GMC estimates that 30% of unsatisfied demand for Lyras is diverted to demand for Libras and 5% to demand for Hydras. Similarly, 10% of unsatisfied demand for Libras is diverted to demand for Hydras. For example, if the demand for Lyras is 1,400,000 cars, then the unsatisfied demand will be 400,000 if no capacity is added. Out of this unsatisfied demand,
will materialize as demand for Libras, and will materialize as demand for Hydras. Similarly, if the demand for Libras is 1,220,000 cars (1,100,000 original demand plus 120,000 demand diverted from Lyras), then the unsatisfied demand for Lyras would be 420,000 if no capacity is added. Out of this unsatisfied demand, 42,000 will materialize as demand for Hydras. All other unsatisfied demand is lost to competitors.

-- *Practical Management Science*

In [ ]:
!pip install -q gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 60.3 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
print(gp.gurobi.version())

(11, 0, 3)


In [1]:
import gurobipy as gp
from gurobipy import GRB
from itertools import product
import numpy as np

In [2]:
# Define the input data for plants, their capacities, fixed costs, profit margins, and demand
DATA = {
    "Plants": {
        "Lyra": {
            "Capacity": 1000 * 1000,
            "Fixed Cost": 2000 * 1000000,
            "Plant Profit Margin":
                {
                    "Lyra": 2 * 1000,
                    "Libra": 0,
                    "Hydra": 0
                }
        },
        "Libra": {
            "Capacity": 800 * 1000,
            "Fixed Cost": 2000 * 1000000,
            "Plant Profit Margin":
                {
                    "Lyra": 0,
                    "Libra": 3 * 1000,
                    "Hydra": 0
                }
        },
        "Hydra": {
            "Capacity": 900 * 1000,
            "Fixed Cost": 2600 * 1000000,
            "Plant Profit Margin":
                {
                    "Lyra": 0,
                    "Libra": 0,
                    "Hydra": 5 * 1000
                }
        },
        "New Lyra": {
            "Capacity": 1600 * 1000,
            "Fixed Cost": 3400 * 1000000,
            "Plant Profit Margin":
                {
                    "Lyra": 2.5 * 1000,
                    "Libra": 3 * 1000,
                    "Hydra": 0
                }
        },
        "New Libra": {
            "Capacity": 1800 * 1000,
            "Fixed Cost": 3700 * 1000000,
            "Plant Profit Margin":
                {
                    "Lyra": 2.3 * 1000,
                    "Libra": 3.5 * 1000,
                    "Hydra": 4.8 * 1000
                }
        }
    },
    "Demand": {
        "Lyra": 1400 * 1000,
        "Libra": 1100 * 1000,
        "Hydra": 800 * 1000
    },
    "Demand Diversion": {
        "Lyra": {
            "Libra": .3,
            "Hydra": .05
        },
        "Libra": {
            "Lyra": 0,
            "Hydra": .1
        },
        "Hydra": {
            "Lyra": 0,
            "Libra": 0
        }
    }
}

In [3]:
# Define sets of plants and cars for reference
plants = DATA["Plants"].keys()
cars = DATA["Demand"].keys()
productions = [(plant, car) for plant in plants for car in cars]

# Create a new model
m = gp.Model('GMC')

# Define variables for plant operation (binary), production amounts, and adjusted demand
O = m.addVars(plants, vtype=GRB.BINARY, name="Plant")
P = m.addVars(productions, vtype=GRB.INTEGER, name="Production", lb=0.0)
D = m.addVars(cars, vtype=GRB.INTEGER, name="Demand", lb=0.0)


# Constraint: Prevent Lyra and Libra from being open alongside their retooled versions
for plant in ["Lyra", "Libra"]:
    m.addConstr(O[f"{plant}"] + O[f"New {plant}"] <= 1, name=f"Retool_{plant}")

# Constraint: Limit total number of operational plants to exactly 3
m.addConstr(gp.quicksum(O[plant] for plant in plants) == 3, name="Total_Plants")

# Constraint: Require Hydra plant to be operational initially (can be removed later)
m.addConstr(O["Hydra"] == 1, name="Hydra_open")

# Constraint: Set capacity limits for each plant based on its operational status
m.addConstrs(
    (gp.quicksum(P[(plant, car)] for car in cars) <= O[plant] * DATA["Plants"][plant]["Capacity"] for plant in plants),
    name="Capacity"
)

# Constraint: Define adjusted demand for each car type, considering demand diversion
m.addConstrs(
    (D[car] == DATA["Demand"][car] + gp.quicksum(
        DATA["Demand Diversion"][alternative][car] * (DATA["Demand"][alternative] - gp.quicksum(P[(plant, alternative)] for plant in plants))
        for alternative in DATA["Demand Diversion"][car].keys()
    ) for car in cars), name="Adjusted_Demand"
)

# Constraint: Ensure production meets the adjusted demand for each car type
m.addConstrs(
    (gp.quicksum(P[(plant, car)] for plant in plants) <= D[car] for car in cars),
    name="Demand_Fulfillment"
)

# Add constraints to ensure each plant only produces specific car types
for plant in plants:
    for car in cars:
        # If the plant's profit margin for a car is 0, it can't produce that car
        if DATA["Plants"][plant]["Plant Profit Margin"][car] == 0:
            m.addConstr(P[(plant, car)] == 0, name=f"{plant}_Only_{car}")

# Objective: Maximize total profit by subtracting fixed costs from total profit margin
m.setObjective(
    gp.quicksum(P[(plant, car)] * DATA["Plants"][plant]["Plant Profit Margin"][car] for (plant, car) in productions) -
    gp.quicksum(O[plant] * DATA["Plants"][plant]["Fixed Cost"] for plant in plants), GRB.MAXIMIZE
)

# Solve the initial model with Hydra open
m.optimize()

# Helper function to display solution values and objective
def get_solution(model):
    for v in model.getVars():
            print(f"{v.VarName}: {v.X}")
    print(f"Optimal objective value: ${round(model.objVal, 2)}")

# Display initial solution
print("\n--- Initial Optimization Solution with Hydra Open ---")
get_solution(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-16
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 22 rows, 23 columns and 73 nonzeros
Model fingerprint: 0x320ed790
Variable types: 0 continuous, 23 integer (5 binary)
Coefficient statistics:
  Matrix range     [5e-02, 2e+06]
  Objective range  [2e+03, 4e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective -3.41600e+09
Presolve removed 14 rows and 11 columns
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 29 nonzeros
Variable types: 0 continuous, 12 integer (2 binary)
Found heuristic solution: objective -2.10000e+09

Roo

In [ ]:
def get_production(car):
    production = 0
    for x in P.keys():
        if x[1] == car:
            production += P[x].X
    return production

* The decision variables in this model include:

Binary Variables (O): Determines if a plant is open, with one variable for each of the 5 plant options (Lyra, Libra, Hydra, New Lyra, New Libra).

Production Variables (P): Defines production levels, with one variable for each car type at each plant.

Demand Variables (D): Defines demand for each car type.


In [ ]:
num_decision_vars = len(O) + len(P) + len(D)
print(f"Total number of decision variables: {num_decision_vars}")


Total number of decision variables: 23


* Assumption that Intentional under-production (unmet demand) for Lyra's is successful in driving more demand for the more expensive car models is true or not. We’ll calculate the unmet demand for Lyras and see how it diverts to Libras and Hydras.
    

In [ ]:
# Calculate total production for Lyra
lyra_production = sum(P[(plant, "Lyra")].X for plant in plants)
lyra_demand = DATA["Demand"]["Lyra"]
unmet_lyra_demand = lyra_demand - lyra_production

# Calculate diverted demand based on diversion rates
diverted_to_libra = unmet_lyra_demand * DATA["Demand Diversion"]["Lyra"]["Libra"]
diverted_to_hydra = unmet_lyra_demand * DATA["Demand Diversion"]["Lyra"]["Hydra"]

print(f"Unmet Lyra Demand: {unmet_lyra_demand}")
print(f"Diverted to Libra: {diverted_to_libra}")
print(f"Diverted to Hydra: {diverted_to_hydra}")


Unmet Lyra Demand: 142850.0
Diverted to Libra: 42855.0
Diverted to Hydra: 7142.5


    * Lets explore if GMC should retool the Libra plant

In [ ]:
# Force retooling of the Libra plant
m.addConstr(O["New Libra"] == 1, "Force_Retool_Libra")
m.optimize()

# Evaluate if retooling the Libra plant results in a higher profit
print(f"Objective value after retooling Libra: ${round(m.objVal, 2)}")
get_solution(m)


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 23 columns and 67 nonzeros
Model fingerprint: 0x69a36d3b
Variable types: 0 continuous, 23 integer (5 binary)
Coefficient statistics:
  Matrix range     [5e-02, 2e+06]
  Objective range  [2e+03, 4e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint Force_Retool_Libra by 1.000000000

Found heuristic solution: objective -3.41600e+09
Presolve removed 8 rows and 10 columns
Presolve time: 0.00s
Presolved: 8 rows, 13 columns, 34 nonzeros
Variable types: 0 continuous, 13 integer (1 bina

* Lets Calculating additional demand for Hydras due to unmet demands from Lyras and Libras.

In [ ]:
# Calculate additional demand driven for Hydras
lyra_to_hydra = unmet_lyra_demand * DATA["Demand Diversion"]["Lyra"]["Hydra"]
libra_production = sum(P[(plant, "Libra")].X for plant in plants)
unmet_libra_demand = DATA["Demand"]["Libra"] - libra_production
libra_to_hydra = unmet_libra_demand * DATA["Demand Diversion"]["Libra"]["Hydra"]

additional_hydra_demand = lyra_to_hydra + libra_to_hydra
print(f"Additional Hydra Demand: {additional_hydra_demand}")


Additional Hydra Demand: 7142.5


* Lets explore if GMC can close the Hydra plant

In [ ]:
# Remove the "open Hydra plant" constraint and re-optimize
hydra_constraint = m.getConstrByName("Hydra_open")
if hydra_constraint:
    m.remove(hydra_constraint)
    m.update()  # Update model before re-optimization

m.optimize()
print("\n--- After Closing the Hydra Plant ---")
get_solution(m)


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 15 rows, 23 columns and 66 nonzeros
Model fingerprint: 0x95bd057c
Variable types: 0 continuous, 23 integer (5 binary)
Coefficient statistics:
  Matrix range     [5e-02, 2e+06]
  Objective range  [2e+03, 4e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolved: 8 rows, 13 columns, 34 nonzeros

Continuing optimization...


Explored 1 nodes (8 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 2.56e+09 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.560000000000e+09, best bound 2.560000000000e+09, gap 0.0000%

--- After 

* If Libra demand changes to 2,000,000. Will retooling plan change?

In [ ]:
# Update Libra demand
DATA["Demand"]["Libra"] = 2000 * 1000
m.update()  # Ensures data update in the model
m.optimize()
retooled_libra = O["New Libra"].X == 1
print("\n--- After Increasing Demand for Libra to 2,000,000 ---")
print(f"Does the retooling plan change? {'Yes' if retooled_libra else 'No'}")
get_solution(m)


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 15 rows, 23 columns and 66 nonzeros
Model fingerprint: 0x95bd057c
Variable types: 0 continuous, 23 integer (5 binary)
Coefficient statistics:
  Matrix range     [5e-02, 2e+06]
  Objective range  [2e+03, 4e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolved: 8 rows, 13 columns, 34 nonzeros

Continuing optimization...


Explored 1 nodes (8 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 2.56e+09 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.560000000000e+09, best bound 2.560000000000e+09, gap 0.0000%

--- After 